In [ ]:
%%time
%matplotlib inline
from new_import import *

In [ ]:
%%time
# Dask gateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,4))
dc = datacube.Datacube()

# Configure s3 access
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client)

client

In [ ]:
## cấu hình thời gian lấy ảnh và tọa độ
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.86575, 105.94120)
latitude_range = (9.65070, 9.69850)

In [ ]:
## truy vấn ảnh vệ tinh sen2
data = load_data(dc, date_range, longtitude_range, latitude_range)
notebook_utils.heading(notebook_utils.xarray_object_size(data))
display(data)

In [ ]:
%%time
# Tiến hành loại bỏ các vị trí bị mây ảnh hưởng
result = mask_clean(data)
progress(result)

In [ ]:
# Tiến hành tính toán NDVI
ds1 = calculate_indices(result, index='NDVI', satellite_mission='s2')
ndvi = ds1["NDVI"]
display(ndvi)

In [ ]:
## ảnh ndvi chưa fill nan
plt.imshow(ndvi.isel(time=50))

In [ ]:
# đặt thời gian các mùa
time_split = [slice('2022-09-01', '2023-01-01'), 
               slice('2023-01-01', '2023-05-01'),
               slice('2023-05-01', '2023-07-01'),
               slice('2023-07-01', '2023-10-01')]

# fill nan
fill_nan_ndvi = fill_nan(ndvi, time_split)

## ảnh ndivi đã fill nan
plt.imshow(fill_nan_ndvi.isel(time=50))

In [ ]:
%%time
## tính ndvi theo tháng
average_ndvi = fill_nan_ndvi.resample(time='1M').mean().persist()
progress(average_ndvi)

In [ ]:
# compute average_ndvi
average_ndvi = average_ndvi.compute()

## cấu hình shapefile ranh giới thuận hòa và vh vv file
thuanhoa_path = "ThuanHoa/region/ST_ThuanHoa_Boundaryofficially.shp"
name_vh = "ThuanHoa/ThuanHoa_VH.tif"
name_vv = "ThuanHoa/ThuanHoa_VV.tif"

# load dữ liệu sen1
dsvh, dsvv = load_sen1(name_vh, name_vv)

In [ ]:
# load model RF
loaded_model = joblib.load(os.path.join("model_train", "model.joblib"))

# dự đoán
data_array = predict(loaded_model, data.rio.crs, average_ndvi, dsvh, dsvv)

In [ ]:
# cấu hình màu cho các loại đất
colors = [
    "#abcee9",
    "#ffef44",
    "#c4ff9e",
    "#ffd6a8",
    "#93ddda",
    "#1aeef7",
    "#ffa7f2",
    "#33ee33"
]
labels = [
    "Lúa tôm",
    "Lúa",
    "CHN",
    "CLN",
    "TS",
    "Sông",
    "Đất xây dựng",
    "Rừng"
]

In [ ]:
# hiển thị phân loại sử dụng đất
cmap = ListedColormap(colors)
img = data_array.plot(cmap=cmap, add_colorbar=False)
cbar = plt.colorbar(img)
cbar.ax.set_yticklabels(labels)
plt.title("Phân loại sử dụng đất")
plt.axis('off')
plt.show()

In [ ]:
# cắt theo ranh giới xã thuận hòa
region_result = cut_according_shp(thuanhoa_path, average_ndvi, data_array)

In [ ]:
# hiển thị kết quả phân loại sử dụng đất
colorval = list(range(len(colors)))
options = {
    'title': 'Phân loại sử dụng đất',
    'cmap': colors,
    'clim': (0, 8),
    'aspect': 'equal',
    'colorbar_opts': {
        'major_label_overrides': dict(zip(colorval, labels)),
        'major_label_text_align': 'left',
        'ticker': FixedTicker(ticks=colorval),
    },
}
                                      
region_result.hvplot(
    rasterize = True,                        # Use Datashader, particularly useful for dask arrays
    aggregator = reductions.mode(),          # Datashader selects mode value, requires 'hv.Image'
).options(opts.Image(**options))

In [ ]:
# Lưu lại kết quả
region_result.rio.to_raster("KetQuaPhanLoaiDat.tif")

In [ ]:
# đóng client, cluster
client.close()
cluster.close()